In [218]:
def remove_white_space (code):
    code_without_white_space = []
    
    for line in code:
        line = line.split('\n', 1)[0]
        line = line.split('//', 1)[0]
        line = line.strip()
        code_without_white_space.append(line)
    
    code_without_white_space = list(filter(None, code_without_white_space))
    
    return code_without_white_space

def convert_to_binary (number):
    
    binary_number = "{0:b}".format(int(number))
    binary_number =  binary_number.zfill(16)
    return binary_number

def translate_commands (code, label_symbols, variables):
    code_translated = [type_A_commands(line, label_symbols, variables) if line.find('@') != -1 else type_B_commands(line) for line in code]
    
    return code_translated

def type_A_commands (command, label_symbols, variables):
    a_command = command.split('@', 2)[1]
    pre_defined_symbols = {'R0': 0,
                           'R1': 1,
                           'R2': 2,
                           'R3': 3,
                           'R4': 4,
                           'R5': 5,
                           'R6': 6,
                           'R7': 7,
                           'R8': 8,
                           'R9': 9,
                           'R10': 10,
                           'R11': 11,
                           'R12': 12,
                           'R13': 13,
                           'R14': 14,
                           'R15': 15,
                           'SCREEN': 16384,
                           'KBD': 24576,
                           'SP': 0,
                           'LCL': 1,
                           'ARG': 2,
                           'THIS': 3,
                           'THAT': 4}

    

    if a_command.isnumeric():
        a_command = convert_to_binary(a_command)

    elif a_command in pre_defined_symbols:
        a_command = pre_defined_symbols[a_command]
        a_command = convert_to_binary(a_command)

    elif a_command in label_symbols:
        a_command = label_symbols[a_command]
        a_command = convert_to_binary(a_command)

    elif a_command in variables:
        a_command = variables[a_command]
        a_command = convert_to_binary(a_command)      
    else:
        open_space = len(variables)
        variables[a_command] = 16 + open_space
        a_command = variables[a_command]
        a_command = convert_to_binary(a_command)

    return a_command

def type_B_commands (command):
    d = {'M': '001',
         'D': '010',
         'MD': '011',
         'A': '100',
         'AM': '101',
         'AD': '110',
         'AMD': '111'}
    
    c = {'0': '0101010',
         '1': '0111111',
         '-1':'0111010',
         'D': '0001100',
         'A': '0110000',
         'M': '1110000',
         '!D':'0001101',
         '!A':'0110001',
         '!M':'1110001',
         'D+1':'0011111',
         'A+1':'0110111',
         'M+1':'1110111',
         'D-1':'0001110',
         'A-1':'0110010',
         'M-1':'1110010',
         'D+A':'0000010',
         'D+M':'1000010',
         'D-A':'0010011',
         'D-M':'1010011',
         'A-D':'0000111',
         'M-D':'1000111',
         'D&A':'0000000',
         'D&M':'1000000',
         'D|A':'0010101',
         'D|M':'1010101'}

    j = {'JGT': '001',
         'JEQ': '010',
         'JGE': '011',
         'JLT': '100',
         'JNE': '101',
         'JLE': '110',
         'JMP': '111'}

    binary_code = '111'

    if command.find('=') != -1:
        command_d = command.split('=', 2)[0]
        not_translated = command.split('=', 2)[1]
        
        if command_d in d:
            command_d = d[command_d]
    else:
        command_d = '000'
        not_translated = command
    
    if not_translated.find(';') != -1:
        command_c = not_translated.split(';', 2)[0]
        command_j = not_translated.split(';', 2)[1]

        if command_c in c:
            command_c = c[command_c]
        if command_j in j:
            command_j = j[command_j]
    else:
        if not_translated in c:
            command_c = c[not_translated]

        command_j ='000'

    binary_code += command_c + command_d + command_j
    return binary_code

def get_label_symbols(code, label_symbols):
    code_without_symbols = []
    
    for i in range(len(code)):
        if code[i].find('(') != -1:
            label_symbols[code[i][1:-1]] = len(code_without_symbols)
        else:
            code_without_symbols.append(code[i])
            
    return code_without_symbols

In [222]:
directory = 'add/'
file_name = 'Add.asm'

with open(directory + file_name) as f:
    code = f.readlines()

label_symbols = {}
variables = {}

code = remove_white_space(code)
code = get_label_symbols(code, label_symbols)
binary_code = translate_commands(code, label_symbols, variables)

textfile = open(file_name[:-3] + "hack", "w")
for element in binary_code:
    textfile.write(element + "\n")
textfile.close()